# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686548


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:41,  1.48s/it]

Rendering models:  13%|█▎        | 4/30 [00:03<00:27,  1.06s/it]

Rendering models:  20%|██        | 6/30 [00:03<00:18,  1.30it/s]

Rendering models:  27%|██▋       | 8/30 [00:03<00:12,  1.78it/s]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:02,  3.60it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:01,  4.63it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  5.95it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  6.89it/s]

citystars                             0.000963
hailstorm3812                         0.000310
MissDonkey                            0.000177
Camelron                              0.000128
mkehrli                               0.000135
Roquwat                               0.000376
rnevils                               0.000158
nsyee                                 0.000283
justsomenoodles                       0.000290
jmfranci                              0.000807
not-logged-in-0a9937b931b705126743    0.000129
not-logged-in-5f30b2bb9b7f9d49da41    0.001452
coyotes54                             0.000161
AlexYoung35                           0.000257
acapirala                             0.000175
Ellaaa666                             0.064402
Lavadude                              0.023799
not-logged-in-b79145bb64292e223990    0.000358
nicholas.                             0.013225
h4ranek                               0.001035
tingard                               0.000130
Andruha_T    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())